# Lab exercise 2: Regression, Least squares, QR

## Part 4: QR factorization and application to least squares problem


This part of the lab the QR factorization in Python, and its application for solving the least squares problem.


### QR factorization

Every matrix $A\in \mathbb R^{m\times n}$ with $m\geqslant n$ (overdetermined) can be factorized as
$$
A = QR
$$
where $Q$ is a $m\times m$ **orthogonal** matrix and $R$ is a $m\times n$ **upper triangular** matrix.

$$
\begin{array}{cccc}
\begin{pmatrix}
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times
\end{pmatrix}&=&
\begin{pmatrix}
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times
\end{pmatrix}&
\begin{pmatrix}
\times & \times  & \times\\
 0 & \times  & \times\\
 0 &  0  & \times \\ \hline
 0 & 0 & 0\\
  0 & 0 & 0
\end{pmatrix}
\\ \\
A&& Q&R
\end{array}
$$

As we see $m-n$ last rows of $R$ are zero which role out the last $m-n$ columns of $Q$, enable us to write the thin (or reduced) QR factorization
$$
A = Q_1 R_1
$$
where $Q = [Q_1\;Q_2]$ and $R = \begin{bmatrix} R_1\\ 0 \end{bmatrix}$. Here $Q_1$ consists of the first $n$ columns of $Q$.

In numpy, the command `Q, R = np.linalg.qr(A, mode = 'complete')` computes the complete QR factorization of the matrix $A$. As an example, in the cell below compute and print the Q and R factors of the following matrix:
$$
A = \begin{bmatrix}
1&-1&1\\
1&1&1\\
1&2&4\\
1&3&9\\
1&4&16
\end{bmatrix}
$$
This is the coefficient matrix of a quadratic polynomial fitting on points $-1,1,2,3,4$.

In [ ]:
# enter your code here


Changing the `mode` to `mode = 'reduced'` gives the reduced QR factorization. Compute and print the reduced QR factorization of the above matrix in the cell below:

In [ ]:
# enter your code here



### Computing $Q$ and $R$ using Householder reflections

Given a non-zero vector $u$ in $\mathbb R^n$, a matrix of the form
\begin{equation*}
  H = I -\frac{2}{u^Tu}uu^T
\end{equation*}
is called a **Householder matrix** or a **Householder reflection**.

Matrix $H$ is orthogonal and can be used to iteratively introduce zeros under the diagonals of matrix $A$ ultimately leading to the QR factorization of $A$. Let's illustrate this process with an example:

Consder the matrix $A$ above and let $x$ be its first column, i.e.
$$
x = \begin{bmatrix}
1\\ 1\\ 1\\ 1\\ 1
\end{bmatrix}
$$
Now set $u = x + \mathrm{sign}(x_1)\|x\|_2 e_1$ where $e_1 = [1,0,0,0,0]^T$, and then compute the Householder matrix $H_1 = I-(2/u^Tu)uu^T$.
The code is given below:

In [45]:
import numpy as np

def HouseVec(x):
  e = np.zeros(len(x)); e[0] = 1
  s = -1 if x[0] < 0  else 1      # our sign function (it is different from np.sign(a) as np.sign(0) = 0)
  u = x + s*np.linalg.norm(x,2)*e
  return u

A = np.array([[1,-1,1],[1,1,1],[1,2,4],[1,3,9],[1,4,16]])
x = A[:,0] # first column
n = len(x)
u = HouseVec(x)
H1 = np.eye(n) - 2/np.dot(u,u) * np.outer(u,u)

Multiply $H_1$ by $A$ to get $A_1$: (for matrix-matrix or matrix-vector multiplication you can use `@` operator or `np.matmul()`, or `np.dot()` commands. Avoid using `*` operator.)

In [ ]:
# enter your code here



What does $A_1$ look like? For the second step, let's assume that $x$ is a $4\times 1$ vector located below the diagonal in the second column of $A_1$, including the diagonal elements. Now, proceed with the same procedure to compute the Householder matrix $\widetilde{H}_2$. Finally, construct the matrix $H_2$ as follows:
$$
H_2 = \begin{bmatrix}
1& 0 \\ 0 & \widetilde H_2
\end{bmatrix}.
$$
The code is given below:

In [43]:
x = A1[1:,1]  # second column diagonal and under diagonal
u = HouseVec(x)
Ht = np.eye(n-1) - 2/np.dot(u,u) * np.outer(u,u)
H2 = np.block([[np.ones(1), np.zeros([1,n-1])],[np.zeros([n-1,1]), Ht]])

Note that `np.block` is used to form a block matrix. It works similar to `np.array` by replacing matrices instead of individual numbers.

Now compute and print the new matrix $A_2 = H_2A_1$ in the cell below. What do you observe?

In [44]:
# enter your code here



Continue with this process to the third step and calculate matrix $A_3$ as the product of $H_3$ and $A_2$. It's important to note that this resulting matrix should be upper triangular, representing the $R$ factor in the QR factorization. The $Q$ factor is obtained as the product of $H_1$, $H_2$, and $H_3$. Finalize your code and compute both factors accordingly:

In [ ]:
# enter your code here


Compare your computed factors $Q$ and $R$ with those of the `np.linalg.qr` command:

In [ ]:
# enter your code here



### Application to solve the least squares problem

For a full-rank matrix $A$, the solution of the least square problem
$$
\mbox{find}~~ x\in\mathbb R^n ~~\mbox{such that }~~ \|Ax-b\|_2 ~~\mbox{is mimimized}
$$
is obtained by solving the triangular system
$$R_1x = Q_1^Tb$$
using a backward substitution, and the residual is
$$residual = \|Ax-b\|_2=\|Q_2^Tb\|_2.$$
Assume that $A$ and $b$ are given as
$$
A = \begin{bmatrix}
1&-1&1\\
1&1&1\\
1&2&4\\
1&3&9\\
1&4&16
\end{bmatrix}
,\qquad
b = \begin{bmatrix}
1\\ 2\\ 1\\ 2\\ 3
\end{bmatrix}.
$$
Write a code to compute the least square solution of $Ax\approx b$ using the QR factorization and compute the residual:

In [ ]:
# enter your code here

